# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>
If you want to skip this step, you can follow the steps below the feature extraction to load the preprocessed data, 
retrain Sherlock and generate predictions.

In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import tensorflow as tf

from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction
from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock

### matplotlib, numpy, pandas, scikit-learn, tensorflow, googledrivedownloader, nltk, gensim, pyarrow, tqdm, -e .

## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [39]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


## Read in raw data
You can skip this step if you want to use a preprocessed data file.

In [152]:
train_samples = pd.read_parquet('../data/data/raw/train_values.parquet')
train_labels = pd.read_parquet('../data/data/raw/train_labels.parquet')

In [99]:
validation_samples = pd.read_parquet('../data/data/raw/val_values.parquet')
validation_labels = pd.read_parquet('../data/data/raw/val_labels.parquet')

In [116]:
test_samples = pd.read_parquet('../data/data/raw/test_values.parquet')
test_labels = pd.read_parquet('../data/data/raw/test_labels.parquet')

In [100]:
validation_samples.head()

,values
217858,"['Shelby', 'Knox', 'Washington', 'Davidson', '..."
166650,"['CDs, Adult Musical', 'CDs, Adult Musical', '..."
27611,"[20.42, 23.17, 24.5, 24.0, 21.92, 28.08, 25.5,..."
370305,"['Nick Scholfield', 'R Johnson', 'P J Brennan'..."
45377,"['', 'Tres Hombres (Expanded & Remastered)', '..."


In [102]:
validation_labels.head()

,type
217858,county
166650,collection
27611,age
370305,jockey
45377,album


## Extract features
It is important that the string-representations of lists are first converted into lists of strings.
The labels should be a list of semantic types.

In [178]:
test_samples_converted, y_test = convert_string_lists_to_lists(test_samples, test_labels, "values", "type")

100%|██████████| 137353/137353 [03:37<00:00, 631.92it/s] 


In [149]:
test_samples_converted.head()

20368     [Central Missouri, unattached, unattached, Kan...
664102    [95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, ...
366813    [Katie Crews, Christian Hiraldo, Alex Estrada,...
530567    [Christian, Non-Christian, Unreported, Jewish,...
176253    [AAF-McQuay Canada Inc., AAF-McQuay Canada Inc...
Name: values, dtype: object

### Given that feature extraction can take long, we only take the first 100 samples.

In [179]:
y_test_subset = y_test[:100]

In [173]:
X_test = extract_features(test_samples_converted.head(n=100))

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Extracting features for data column: 100


In [180]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,False,False,0.000000,0.000000,0,0,0.0,0,-3.0,0.000000,...,0.000350,-0.001248,0.000052,0.000876,-0.000085,0.000608,0.000733,0.000788,0.000388,0.000395
1,True,False,0.285714,0.204082,0,1,0.0,2,-1.1,0.948683,...,-0.000382,-0.001030,-0.000536,0.000089,-0.000298,0.000654,0.000786,-0.000864,0.001170,-0.000153
2,False,False,0.000000,0.000000,0,0,0.0,0,-3.0,0.000000,...,0.001025,-0.000692,0.000178,0.000489,-0.001189,-0.000285,-0.001204,-0.001224,-0.000400,-0.000449
3,False,False,0.000000,0.000000,0,0,0.0,0,-3.0,0.000000,...,0.001063,-0.000571,0.000511,0.001078,-0.000766,-0.000471,0.000906,-0.000896,-0.000938,0.001191
4,False,False,0.000000,0.000000,0,0,0.0,0,-3.0,0.000000,...,-0.000154,0.000801,0.000231,0.000362,0.000722,0.000854,0.000432,-0.001082,0.000645,0.001041


## Impute NaN values with feature means

In [135]:
# train_columns_means = pd.DataFrame(X_train.mean()).transpose()

In [145]:
# X_train.fillna(train_columns_means.iloc[0], inplace=True)
# X_validation.fillna(train_columns_means.iloc[0], inplace=True)
# X_test.fillna(train_columns_means.iloc[0], inplace=True)

## Retrain sherlock
The model can be retrained using the code below. The model is currently restricted to be trained on 78 classes, please check the README for more details and a work-around.

In [148]:
train_sherlock(X_train, y_train, X_val, y_val, nn_id='retrained_sherlock');
print('Trained and saved new model.')

## Generate predictions with a model
If you want to use the pretrained Sherlock model `nn_id` set to "sherlock".

If you want to use another model, you can use the identifier corresponding to that model.

**Note**: There is a bug somewhere in the refactored code which affects the model predictions, this should be fixed soon.

In [184]:
predicted_labels = predict_sherlock(X_test, nn_id='sherlock')

In [186]:
f1_score(y_test_subset, predicted_labels, average="weighted")

0.5753809523809523

## Generate predictions with preprocessed data using Sherlock

Requires the data to be downloaded from Google Drive (see first step in notebook).

In [36]:
X_test_preprocessed = pd.read_parquet("../data/data/processed/X_test.parquet")
y_test_preprocessed = pd.read_parquet("../data/data/processed/y_test.parquet").reset_index(drop=True)

In [25]:
X_test_preprocessed.head()

,col_entropy,frac_unique,frac_numcells,frac_textcells,avg_num_cells,std_num_cells,avg_text_cells,std_text_cells,avg_spec_cells,std_spec_cells,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,2.122181,0.005,0.000,1.0,0.000,0.000000,12.290,5.077194,0.0,0.0,...,0.023563,-0.029472,0.002835,0.090851,-0.125505,-0.027747,0.028412,-0.078901,0.054292,-0.049115
1,3.817487,0.015,1.000,0.0,2.058,0.233743,0.000,0.000000,0.0,0.0,...,0.244085,-0.055574,0.017600,0.079978,-0.014825,0.006086,0.121871,-0.078689,-0.069111,-0.112550
2,3.166061,0.009,0.120,1.0,0.120,0.324962,11.527,2.688730,0.0,0.0,...,0.018266,-0.088117,-0.048036,-0.011286,-0.109643,-0.070223,-0.009666,-0.081991,-0.041528,-0.094458
3,2.316887,0.005,0.000,1.0,0.000,0.000000,9.053,1.960151,0.0,0.0,...,-0.063415,-0.000197,0.012020,-0.033859,0.063092,0.075499,-0.009511,-0.070606,0.061907,0.065065
4,6.955528,0.163,0.018,1.0,0.072,0.531804,20.268,9.593132,0.0,0.0,...,0.015399,-0.213604,0.029100,-0.009626,-0.154028,-0.090470,-0.013950,0.036592,-0.139673,-0.115430


In [193]:
y_test_preprocessed.head()

,label
index,
511600,affiliation
146358,weight
665579,jockey
148486,religion
3546,company


In [194]:
predicted_labels = predict_sherlock(X_test_preprocessed.head(n=25), 'sherlock')

In [195]:
f1_score(y_test_preprocessed.head(n=25), predicted_labels, average='weighted')

0.8906666666666667